In [1]:
import pandas as pd
import os 
from functools import reduce

In [2]:
folder_path = '/Users/AnhHuynh/Documents/CS504-006-Team2/data/monthly_data'  # Replace with your path
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

csv_dfs = []

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)

    # Convert to datetime
    df['observation_date'] = pd.to_datetime(df['observation_date'])

    csv_dfs.append(df)

In [3]:
# house_index = pd.read_csv("data/USSTHPI-quarterly.csv")
recession_indicator = pd.read_csv("data/USRECQ-quarterly.csv")
gdp = pd.read_csv("data/GDPC1-quarterly.csv")

In [4]:
def expand_quarterly_to_monthly(df, value_col):
    """
    Expand a quarterly dataset to monthly frequency by evenly distributing the value
    over the 3 months in each quarter.

    Parameters:
        df (pd.DataFrame): Input DataFrame with 'observation_date' and value column.
        value_col (str): Name of the column containing quarterly values.

    Returns:
        pd.DataFrame: Expanded monthly DataFrame.
    """
    df['observation_date'] = pd.to_datetime(df['observation_date'])

    def expand_row(row):
        base_date = row['observation_date']
        monthly_value = row[value_col] / 3
        return pd.DataFrame({
            'observation_date': [base_date, base_date + pd.DateOffset(months=1), base_date + pd.DateOffset(months=2)],
            value_col: [monthly_value] * 3
        })

    monthly_df = pd.concat([expand_row(row) for _, row in df.iterrows()], ignore_index=True)
    monthly_df.sort_values('observation_date', inplace=True)
    monthly_df.reset_index(drop=True, inplace=True)
    return monthly_df

In [5]:
# house_index_monthly= expand_quarterly_to_monthly(house_index, "USSTHPI")
recession_indicator_monthly = expand_quarterly_to_monthly(recession_indicator, "USRECQ")
gpd_monthly = expand_quarterly_to_monthly(gdp, "GDPC1")

In [6]:
T10Y3M = pd.read_csv("data/T10Y3M-dailycsv.csv")

T10Y3M['observation_date'] = pd.to_datetime(T10Y3M['observation_date'])

# Resample to monthly (you can also use 'sum', 'max', etc.)
T10Y3M_monthly = T10Y3M.resample('MS', on='observation_date').mean().reset_index()

In [8]:
all_dfs = csv_dfs + [recession_indicator_monthly, gpd_monthly, T10Y3M_monthly]

In [9]:
# --- Step 3: Merge all 8 on observation_date ---
merged_df = reduce(lambda left, right: pd.merge(left, right, on='observation_date', how='outer'), all_dfs)

# --- Step 4: Sort and export ---
merged_df.sort_values('observation_date', inplace=True)
merged_df.reset_index(drop=True, inplace=True)

# Output preview
print(merged_df.head(50))


   observation_date  CSUSHPINSA  HOUST  MSACSRNSA  FEDFUNDS        RPI  \
0        2006-01-01     180.828   2273        5.9      4.29  13214.527   
1        2006-02-01     181.500   2119        6.1      4.49  13283.182   
2        2006-03-01     182.749   1969        5.1      4.59  13340.072   
3        2006-04-01     183.648   1821        5.6      4.79  13347.045   
4        2006-05-01     184.380   1942        5.5      4.94  13336.874   
5        2006-06-01     184.547   1802        5.8      4.99  13353.303   
6        2006-07-01     184.607   1737        6.9      5.24  13342.387   
7        2006-08-01     184.404   1650        6.5      5.25  13329.621   
8        2006-09-01     184.198   1720        7.0      5.25  13408.301   
9        2006-10-01     184.053   1491        7.5      5.25  13479.950   
10       2006-11-01     183.630   1570        7.8      5.25  13547.935   
11       2006-12-01     183.229   1649        7.6      5.24  13627.346   
12       2007-01-01     182.718   1409

In [10]:
merged_df.to_csv("combined_dataset.csv", index=False)

In [11]:
rpi = pd.read_csv('data/monthly_data/RPI.csv')

In [21]:
cs_house_index = pd.read_csv("data/monthly_data/CSUSHPINSA.csv")
cs_house_index.head()

,observation_date,CSUSHPINSA
0,2006-01-01,180.828
1,2006-02-01,181.500
2,2006-03-01,182.749
3,2006-04-01,183.648
4,2006-05-01,184.380


In [13]:
data = pd.read_csv("combined_dataset.csv")

data.head()

,observation_date,CSUSHPINSA,HOUST,MSACSRNSA,FEDFUNDS,RPI,PCEPI,USRECQ,GDPC1,T10Y3M
0,2006-01-01,180.828,2273,5.9,4.29,13214.527,83.534,0.0,5451.278333,0.080000
1,2006-02-01,181.500,2119,6.1,4.49,13283.182,83.584,0.0,5451.278333,0.029474
2,2006-03-01,182.749,1969,5.1,4.59,13340.072,83.746,0.0,5451.278333,0.096087
3,2006-04-01,183.648,1821,5.6,4.79,13347.045,84.135,0.0,5465.383667,0.268947
4,2006-05-01,184.380,1942,5.5,4.94,13336.874,84.361,0.0,5465.383667,0.273636


In [14]:
rename_dict = {
    'observation_date': 'Date',
    'CSUSHPINSA': 'house_price_index',
    'HOUST': 'new_private_house_owned',
    'MSACSRNSA': 'home_supply',
    'FEDFUNDS': 'federal_rate',
    'RPI': 'real_income',
    'PCEPI': 'consumption_price_index',
    'USRECQ': 'recession_indicator',
    'GDPC1': 'gdp',
    'T10Y3M': 'treasury_maturity'
}

In [15]:
data = data.rename(columns=rename_dict)

In [16]:
data.head()

,Date,house_price_index,new_private_house_owned,home_supply,federal_rate,real_income,consumption_price_index,recession_indicator,gdp,treasury_maturity
0,2006-01-01,180.828,2273,5.9,4.29,13214.527,83.534,0.0,5451.278333,0.080000
1,2006-02-01,181.500,2119,6.1,4.49,13283.182,83.584,0.0,5451.278333,0.029474
2,2006-03-01,182.749,1969,5.1,4.59,13340.072,83.746,0.0,5451.278333,0.096087
3,2006-04-01,183.648,1821,5.6,4.79,13347.045,84.135,0.0,5465.383667,0.268947
4,2006-05-01,184.380,1942,5.5,4.94,13336.874,84.361,0.0,5465.383667,0.273636
